In [1]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel

# LangChain OpenAI
from langchain_openai import ChatOpenAI
from whisper.utils import system_encoding

In [10]:
## .env 설정

load_dotenv()

True

In [15]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel

# LangChain OpenAI
from langchain_openai import ChatOpenAI

## 이미 생성된 합성 데이터셋 불러오기

In [16]:
## 이미 생성된 합성 데이터셋 생성
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_csv("./ragas_dataset.csv")
print(dataset_df.shape)
display(dataset_df)

(26, 4)


,user_input,reference_contexts,reference,synthesizer_name
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요...",single_hop_specific_query_synthesizer
1,PostgreSQL의 주요 역할은 무엇입니까?,"['### 2.2. 데이터베이스\n- **역할**: 사용자 데이터, 콘텐츠, 로그 ...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다.",single_hop_specific_query_synthesizer
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,['### 2.5. 파일 저장소\n- **역할**: 이미지 및 기타 미디어 파일을 ...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.,single_hop_specific_query_synthesizer
3,다중 인증(MFA)이란 무엇인가요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...,single_hop_specific_query_synthesizer
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,"['### 2.2 비밀번호 관리\n- 비밀번호는 최소 8자 이상, 대문자, 소문자,...","모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링...",single_hop_specific_query_synthesizer
5,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 어떤 절차를 따르...,['## 4. 로그 관리\n\n### 4.1 로그 기록\n- 모든 인증 시도 및 A...,백엔드 개발팀은 로그 관리에서 모든 인증 시도 및 API 호출에 대한 로그를 기록하...,single_hop_specific_query_synthesizer
6,v1.0 버전 에러 핸들링 매뉴얼에서 에러 종류 뭐 있나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"에러는 크게 두 가지로 분류됩니다. 첫째, 클라이언트 에러로 사용자의 요청에 문제가...",single_hop_specific_query_synthesizer
7,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 어떻게 이루어...,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토는 모든 변경 사항에 대해 코드...,single_hop_specific_query_synthesizer
8,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 무엇인가요?,['## 체크리스트\n\n- [ ] 코드 리뷰 완료\n- [ ] 모든 테스트 통과\...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...,single_hop_specific_query_synthesizer
9,데이터베이스 스키마에서 사용자 엔티티는 어떤 정보를 저장하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,사용자(User) 엔티티는 사용자 정보를 저장합니다.,single_hop_specific_query_synthesizer


## 우리가 쓰는 sllm RAG 체인 불러오기

In [1]:
from dotenv import load_dotenv

load_dotenv()

from pathlib import Path
import sys

root = Path.cwd()
while not (root / "utils4").exists() and root != root.parent:
    root = root.parent
sys.path.insert(0, str(root))

from utils4.rag import chat

response = chat({"history": [
    {"role": "user", "content": "안녕!"}], "tone":"informal", "permission":"backend"})

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
INFO:utils4.service:-------- Chat Request - Permission: backend, Tone: informal
INFO:utils4.service:-------- Tools Prompt: 
                                사용자는 backend(백엔드)팀에 속한 팀원입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "backend_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 

In [2]:
print(response)

{'response': '안녕. 뭐 도와줄까?', 'result': ['없음']}


In [3]:
print(response['response'])

안녕. 뭐 도와줄까?


In [12]:
response = chat({"history": [
    {"role": "user", "content": "안녕하세요!"},
    {"role": "assistant", "content": "반가워요!"},
    {"role": "user", "content": "백엔드 팀 문서 알려줘"},
    {"role": "assistant", "content": "백엔드 팀 문서에는 주간 보고서가 있습니다."},
    {"role": "user", "content": "거기엔 무슨 내용이 있어?"},
    {"role": "assistant", "content": """백엔드팀 주간 업무 계획 문서에는 이렇게 적혀 있어.

1. 서비스 안정성 강화: 서버 모니터링 도구 점검, 주요 지표 확인, 이상 징후 대응 방안 마련
2. 신규 기능 개발 및 배포: 요구사항 정의서 작성, API 설계, 개발 일정 수립, 코드 리뷰 및 테스트, 배포 준비
3. 기술 부채 관리 및 리팩토링: 코드 리뷰 일정 수립, 기술 부채 리스트 업데이트

또, 주간 업무 계획에는 서비스 안정성 향상, 신규 기능 개발 로드맵, 기술 부채 관리 등 구체적인 업무 항목과 체크리스트가 포함돼 있어."""},
    {"role": "user", "content": "오늘 저녁 메뉴 추천해줄래?"},
    {"role": "assistant", "content": "오늘 저녁 메뉴로는 균형 잡힌 영양과 맛을 고려하여 닭가슴살 샐러드나 구운 채소와 함께하는 햄버거를 추천드립니다. 건강에도 좋고 간편하게 즐기기에도 적합합니다."},
    {"role": "user", "content": "안뇽"}
    ], "tone":"formal", "permission":"backend"})

INFO:utils4.service:-------- Chat Request - Permission: backend, Tone: formal
INFO:utils4.service:-------- Tools Prompt: 
                                사용자는 backend(백엔드)팀에 속한 팀원입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "backend_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
                </tools>

                For each function call, return a json object with function nam

In [10]:
print(response['response'])

서비스 안정성 향상을 위한 주요 내용은 다음과 같습니다.

1. 서버 모니터링 도구 점검: 현재 사용 중인 모니터링 도구의 상태와 기능을 점검하여 최적화가 필요할 경우 조치합니다.
2. 주요 지표 확인: 서버의 CPU 사용률, 메모리 사용량, 디스크 I/O, 네트워크 대역폭 등 주요 성능 지표를 실시간으로 확인합니다.
3. 이상 징후 대응 방안 마련: 모니터링 과정에서 나타나는 이상 징후에 대해 사전에 대응 방안을 마련하고, 긴급 상황 발생 시 신속히 대응할 수 있도록 준비합니다.

이와 함께 정기적인 서버 점검과 로그 분석을 통해 문제를 사전에 예방하고, 서비스 중단이나 성능 저하를 최소화하는 절차를 수행합니다.


## 정성적 평가 체인 셋팅

In [13]:
def api_document_evaluation_chain_json():
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={"response_format": {"type": "json_object"}},
    )

    evaluation_prompt = PromptTemplate.from_template(
        """
    당신은 코드노바의 사내 문서 기반 평가 전문가입니다. 주어진 시스템의 답변이 **기준 정답과 얼마나 일치하는지**, **중요한 정보를 얼마나 잘 재현했는지**, 그리고 **세부 사항을 얼마나 잘 반영했는지** 평가해야 합니다.

    평가 기준:
    1. **정확도 (Accuracy)**: 시스템의 답변이 **기준 정답**과 얼마나 일치하는지 평가하세요.
        - 100점: **완벽하게 일치**. 구체적인 매개변수, 반환값, 예시까지 정확히 일치.
        - 90-99점: **일치**하나, 예시나 코드에서 **약간 차이**가 있을 때.
        - 70-89점: **핵심 내용**은 맞지만 **세부 내용**이 **누락되거나 잘못된 설명**.
        - 0-69점: **기준 정답과 크게 다름**. 예시나 코드에서 **큰 차이**가 있을 경우 점수는 **0점**.

    2. **재현율 (Recall)**: 시스템의 답변이 **기준 정답에서 제공한 중요한 정보**(예: 매개변수, 반환값 등)를 얼마나 잘 재현했는지 평가하세요.
        - 100점: **모든 핵심 정보**를 **정확하게 재현**.
        - 90-99점: **핵심 정보**는 잘 재현되었으나 **세부 내용**이 부족하거나 다르게 표현됨.
        - 70-89점: **핵심 정보**가 **누락**되었거나 **잘못된 설명**이 있음.
        - 0-69점: **핵심 정보**가 많이 **누락**됨.

    3. **구체성 (Specificity)**: 시스템의 답변이 **기준 정답에서 제공한 세부 사항**(매개변수 설명, 예시 코드 등)을 얼마나 구체적으로 반영했는지 평가하세요.
        - 100점: **매개변수**와 **반환값**을 **구체적으로 설명**하며, **예시 코드**나 **상세한 설명**을 포함.
        - 90-99점: **핵심 매개변수**는 설명되었지만, **구체적인 형식이나 예시 코드**가 부족.
        - 70-89점: **기본적인 설명**은 있지만 **구체적인 예시나 설명**이 부족하거나 불완전.
        - 0-69점: **구체적인 설명**이 부족하고, 중요한 **세부 사항**이 **누락**되었을 때 점수는 **0점**.

    ## 예시 1:
    **질문**: 코드노바의 `API 서버 기술 스택`은 무엇인가요?
    **기준 정답 (코드노바 사내 문서)**: "코드노바의 API 서버는 주로 Python, Django를 사용하며, 데이터베이스는 PostgreSQL을 사용합니다. 서버는 AWS EC2에서 호스팅되며, API 게이트웨이로 Nginx를 사용합니다. 클라우드 서비스는 AWS와 Google Cloud를 활용합니다."
    **시스템 답변**: "코드노바는 Django와 PostgreSQL을 사용하여 API 서버를 운영합니다. AWS와 Google Cloud를 이용하고, Nginx로 트래픽을 관리합니다."

    ### 평가:
    - **정확도 (Accuracy)**: 점수는 0에서 100 사이로 평가하세요. (예: 95)
    - **재현율 (Recall)**: 점수는 0에서 100 사이로 평가하세요. (예: 90)
    - **구체성 (Specificity)**: 점수는 0에서 100 사이로 평가하세요. (예: 80)

    ## 예시 2:
    **질문**: `getDistance` 메서드의 입력 매개변수는 무엇인가요?
    **기준 정답 (코드노바 사내 문서)**: "`getDistance` 메서드는 두 개의 매개변수를 받습니다: `origin` (경로의 시작점, 문자열)과 `destination` (경로의 종료점, 문자열)입니다. 이 메서드는 시작점과 종료점 간의 거리를 계산하여 반환합니다."
    **시스템 답변**: "`getDistance` 메서드는 `origin`과 `destination` 매개변수를 사용하여 거리를 계산합니다."

    ### 평가:
    - **정확도 (Accuracy)**: 점수는 0에서 100 사이로 평가하세요.
    - **재현율 (Recall)**: 점수는 0에서 100 사이로 평가하세요.
    - **구체성 (Specificity)**: 점수는 0에서 100 사이로 평가하세요.

    ## 사용자 질문 : {question}
    ## 기준 정답 : {answer}
    ## 시스템 답변 : {system_answer}

    시스템의 답변을 기반으로 정확도, 재현율, 구체성을 평가한 후, 각 항목에 대한 점수를 **0에서 100 사이**로 계산하여 다음 JSON 형태로 제공합니다:

    {{"accuracy": 85, "recall": 75, "specificity": 65}}
    """
    )

    evaluation_chain = evaluation_prompt | llm | StrOutputParser()
    return evaluation_chain


### 우리의 파인튜닝된 sllm의 RAG 답변을 배치로 정성적 평가

In [17]:
import json
import pandas as pd
import numpy as np

# JSON 응답을 받는 평가 체인 생성
evaluation_chain_json = api_document_evaluation_chain_json()

# 모든 데이터에 대해 평가 수행
evaluation_results = []

for idx in range(len(dataset_df)):
    question = dataset_df['user_input'][idx]
    answer = dataset_df['reference'][idx]
    
    # run_langraph로 시스템 답변 생성
    system_answer = chat({"history": [
    {"role": "user", "content": question}], "tone":"informal", "permission":"cto"})['response']
    
    # 평가 수행
    response = evaluation_chain_json.invoke({
        "question": question,
        "answer": answer,
        "system_answer": system_answer
    })
    
    # JSON 파싱
    try:
        scores = json.loads(response)
        evaluation_results.append({
            'index': idx,
            'question': question,
            'reference': answer,
            'system_answer': system_answer,
            'accuracy': scores.get('accuracy', 0),
            'recall': scores.get('recall', 0),
            'specificity': scores.get('specificity', 0)
        })
        print(f"평가 완료: {idx+1}/{len(dataset_df)} - Accuracy: {scores.get('accuracy', 0)}")
    except json.JSONDecodeError:
        print(f"JSON 파싱 오류: {idx+1}/{len(dataset_df)}")
        evaluation_results.append({
            'index': idx,
            'question': question,
            'reference': answer,
            'system_answer': system_answer,
            'accuracy': 0,
            'recall': 0,
            'specificity': 0
        })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(evaluation_results)

# 평균 점수 계산
avg_accuracy = results_df['accuracy'].mean()
avg_recall = results_df['recall'].mean()
avg_specificity = results_df['specificity'].mean()
overall_avg = (avg_accuracy + avg_recall + avg_specificity) / 3

print(f"\n=== 전체 평가 결과 ===")
print(f"평균 정확도: {avg_accuracy:.2f}")
print(f"평균 재현율: {avg_recall:.2f}")
print(f"평균 구체성: {avg_specificity:.2f}")
print(f"전체 평균: {overall_avg:.2f}")

# 결과 저장
results_df.to_csv('batch_evaluation_results.csv', index=False)
print(f"\n결과가 'batch_evaluation_results.csv'에 저장되었습니다.")


INFO:utils4.service:-------- Chat Request - Permission: cto, Tone: informal
INFO:utils4.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "cto_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
                </tools>

                For each function call, return a json object with function name and arguments within 

평가 완료: 1/26 - Accuracy: 90


INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- LLM Tool Parse Response Success
INFO:utils4.service:-------- LLM Tools Match : 1
INFO:utils4.service:-------- cto search keyword: PostgreSQL의 주요 역할
INFO:utils4.service:-------- cto search found 3 results.
INFO:utils4.service:-------- cto_search tool Response Success
INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- Final Assistant Reply Generated
INFO:utils4.service:-------- PostgreSQL은 주로 사용자 데이터, 콘텐츠, 로그 같은 데이터를 저장하는 역할을 해. 또 데이터베이스 연결 상태랑 쿼리 성능 분석 및 최적화 같은 검증도 중요하게 해.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- Chat Request - Permission: cto, Tone: informal
INFO:utils4.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></t

평가 완료: 2/26 - Accuracy: 80


INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- LLM Tool Parse Response Success
INFO:utils4.service:-------- LLM Tools Match : 1
INFO:utils4.service:-------- cto search keyword: HTTPS 보안 이유
INFO:utils4.service:-------- cto search found 3 results.
INFO:utils4.service:-------- cto_search tool Response Success
INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- Final Assistant Reply Generated
INFO:utils4.service:-------- HTTPS를 쓰는 이유는 데이터 전송 중 보안을 위해 데이터 암호화를 해서 해킹이나 데이터 도청 같은 위협을 막기 위해서야. HTTPS는 HTTPS 프로토콜을 통해 데이터를 암호화해서 전송하는 거야. 이렇게 되면 민감한 정보도 안전하게 보낼 수 있어.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- Chat Request - Permission: cto, Tone: informal
INFO:utils4.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관

...

평가 완료: 24/26 - Accuracy: 90


INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- LLM Tool Parse Response Success
INFO:utils4.service:-------- LLM Tools Match : 1
INFO:utils4.service:-------- cto search keyword: 보안 사고 발생 시 시스템 대응 방법
INFO:utils4.service:-------- cto search found 3 results.
INFO:utils4.service:-------- cto_search tool Response Success
INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- Final Assistant Reply Generated
INFO:utils4.service:-------- 보안 사고가 발생하면 다음과 같은 절차를 따르는 게 좋아.

1. 사고 인식: 시스템 모니터링 도구로 비정상 활동 감지하거나 사용자/팀원 보고를 통해 사고 인식.
2. 사고 분류: 사고 심각도 평가하고 영향을 받는 시스템과 데이터 확인해서 경미, 중간, 심각으로 분류.
3. 초기 대응: 공격 경로 차단하고 시스템 격리.
4. 복구: 백업 데이터로 시스템 복구하고 보안 패치 적용.
5. 데이터 복구: 유출되거나 손상된 데이터 복구하고 영향 받은 사용자에게 통보.
6. 사고 보고서 작성하고 관련자와 공유.
7. 사고 분석 회의 열어서 원인과 문제점 논의하고 예방 대책 마련.

이 절차를 통해 보안 사고를 효과적으로 대응하고 재발 방지할 수 있어.
INFO:httpx:HTTP R

평가 완료: 25/26 - Accuracy: 90


INFO:httpx:HTTP Request: POST https://api.runpod.ai/v2/fcaeo5uwuhd4yj/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils4.service:-------- LLM Tool Parse Response Success
INFO:utils4.service:-------- LLM Tools Match : 0
INFO:utils4.service:-------- Final Assistant Reply Generated
INFO:utils4.service:-------- API 설계는 특정 시스템이나 서비스랑 다른 시스템이나 서비스랑 통신할 수 있는 인터페이스를 설계하는 과정이야. 주로 요청 보내는 방법, 데이터 형식 정의, 응답 처리 방식 같은 걸 포함해. API 설계는 클라이언트랑 서버 간의 상호작용을 명확하고 효율적으로 만들어야 하는 중요한 작업이야. 더 구체적인 내용이나 예시가 필요하면 말해줘.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


평가 완료: 26/26 - Accuracy: 80

=== 전체 평가 결과 ===
평균 정확도: 85.19
평균 재현율: 80.96
평균 구체성: 72.31
전체 평균: 79.49

결과가 'batch_evaluation_results.csv'에 저장되었습니다.


In [12]:
result = pd.read_csv('batch_evaluation_results.csv')

display(result)

,index,question,reference,system_answer,accuracy,recall,specificity
0,0,크랙(Crack) 앱의 주요 기능은 무엇인가요?,"크랙(Crack)은 AI 페르소나 챗봇 앱으로, 코드노바의 서비스 아키텍처에서 생성...","크랙(Crack) 앱의 주요 기능은 생성형 AI를 활용한 글쓰기, 이미지 생성, 요...",85,80,70
1,1,RabbitMQ의 역할은 무엇인가요?,RabbitMQ는 비동기 작업 처리를 위해 시스템 간 메시지를 전달하는 메시지 큐입니다.,"RabbitMQ는 메시지 큐로, 비동기 작업 처리를 위해 시스템 간 메시지를 전달하...",90,95,85
2,2,프론트엔드 팀에서 JWT의 역할은 무엇인가요?,JWT(JSON Web Token)는 사용자 인증을 처리하는 데 사용됩니다.,프론트엔드 팀에서 JWT의 역할에 대한 구체적인 정보는 문서에서 찾을 수 없습니다....,0,0,0
3,3,"코드노바의 백엔드 서비스 아키텍처는 어떻게 설계되었고, 그 목적은 무엇인가요?",코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 ...,"코드노바의 백엔드 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI ...",70,60,50
4,4,MFA는 뭐하는거야?,MFA는 추가적인 보안 강화를 위해 도입되는 다중 인증 방법입니다.,"MFA는 다중 인증(Multi-Factor Authentication)의 약자로, ...",90,95,85
5,5,JWT는 API 접근 시 어떤 역할을 합니까?,"JWT(JSON Web Token)는 API 접근 시 인증을 위해 사용되며, 토큰은...",JWT(JSON Web Token)는 API 접근 시 사용자의 인증을 처리하는 역할...,95,100,90
6,6,코드노바의 백엔드 개발팀이 보안 및 인증을 관리하기 위한 기본 지침은 무엇입니까?,이 가이드는 코드노바의 백엔드 개발팀이 보안 및 인증을 효과적으로 관리하기 위한 기...,코드노바의 백엔드 개발팀이 보안 및 인증을 관리하기 위한 기본 지침은 다음과 같습니...,70,60,80
7,7,2025년 8월 29일에 작성된 코드노바의 에러 핸들링 매뉴얼에서 에러 핸들링의 중...,코드노바의 에러 핸들링 매뉴얼에서는 에러 핸들링이 안정적인 시스템 운영을 위해 필수...,2025년 8월 29일에 작성된 코드노바의 에러 핸들링 매뉴얼에서는 에러 핸들링의 ...,95,95,90
8,8,v1.0 뭐에요?,"v1.0은 코드노바 백엔드팀의 배포 및 운영 가이드의 버전 번호로, 이 문서는 안정...","검색 결과에 따르면, v1.0은 문서의 초기 버전을 의미합니다. 구체적으로는 202...",70,60,50
9,9,코드노바 백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 필요한 체크리스트 항목...,코드노바 백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 필요한 체크리스트 항목...,코드노바 백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 필요한 체크리스트 항목...,100,100,100
